Импортируем необходимые библиотеки

In [1]:
import pandas as pd

import nltk
from nltk.corpus import stopwords as nltk_stopwords
import re 
from nltk.stem import SnowballStemmer

from pymystem3 import Mystem

Откроем и подготовим общий текст

In [2]:
fp = open("kipling.txt")
text = fp.read()
text = text.replace('--', '—') # заменим дефисы на тире

# 1. Поиск имен собственных

Имена собственные начинаются с прописной буквы. Необходимо найти все слова, начинающиеся с заглавной, кроме тех, что являются первым словом в предложении. 

Для начала разобьем текст на предложения.

In [3]:
sentences = re.split(r'[\t\n\r.!?:"…]', text)

Удалим лишние пробелы и тире, а также пустые записи.

In [4]:
for i in range(len(sentences)):
    sentences[i] = sentences[i].strip()
    sentences[i] = sentences[i].strip('— ')
    
sentences = [x for x in sentences if x != '']

Создадим таблицу с именами, порядковыми номерами предложений и номерами слов в предложении.

Добавим одну строку, для корректной работы будущей функции.

In [5]:
names = pd.DataFrame([['Name', '-10', '0']],
                     columns=['name', 'str', 'num'])
names

,name,str,num
0,Name,-10,0


Напишем функцию, которая будет проверять слово на наличие прописной буквы и создавать новую запись (Маугли), либо дополнять предыдущую (Шер + Хан) в зависимости от условий. 

In [6]:
def check(s,n):
    word = sentences[s]                           # Проверяемое слово
    word = word.split()[n]                        
    prev_word = sentences[s].split()[n-1]         # Предыдущее слово в тексте
    if (word[0].isupper()) and (n != 0):          # Проверка регистра и порядкового номера в предложении
        if (
            names.iloc[-1]['str'] == s            # Проверка совпадения номера строки в тексте с последней записью в names
        ) and (
            names.iloc[-1]['num'] == n-1          # Проверка на "соседство" слов
        ) and (
            prev_word[-1] not in [';', ',', ')']  # Проверка на разделители в конце предыдущего слова (Маугли, Багира)
        ) and (
            word[0] != '('                        # Проверка слова на скобку в начале (Багира (Маугли))
        ):
            names.loc[len(names) - 1]['num'] = n  # Переписываем num в последней записи names и дополняем name
            names.loc[len(names) - 1]['name'] = prev_word + ' ' + word
        else:
            names.loc[len(names)] = [word, s, n]  # Создаем новую запись в таблице names

Применим функцию к каждому слову в тексте.

In [7]:
for s in range(len(sentences)):
    for n in range(len(sentences[s].split())):
        check(s,n)

Удалим ненужные символы.

In [8]:
for i in range(len(names)):
    for a in [',', ';', '(', ')']:
        names['name'][i] = names['name'][i].replace(a, '')

Удалим первую строку, которую создавали вручную. 

In [9]:
names = names[1:]

Добавим столбец count для подсчета использования слов в тексте. 

In [10]:
names['count'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Удалим оишние колонки. 

In [11]:
names = names.drop(names.columns[1:3], axis=1).groupby(by='name').sum().sort_values('count', ascending=False).reset_index()

In [12]:
names

,name,count
0,Маугли,173
1,Балу,82
2,Багира,82
3,Рикки-Тикки,69
4,Котик,64
...,...,...
317,Магелланова,1
318,Малаги,1
319,Маленький Братец,1
320,Маленькому Тумаи,1


Мы получили таблицу с именами собственными в разных формах и частотой их употреблений. Необходимо лемматизировать все имена и сгрупировать таблицу.

In [13]:
m = Mystem()

Напишем функцию для лемматизации слова и приведения его в удобную форму.

In [14]:
def lem(word):
    return "".join(m.lemmatize(word)).rstrip('\n').title()

In [15]:
lem_names = names

Применим функцию для лемматизации имен. 

In [16]:
%%time
lem_names['name'] = lem_names['name'].apply(
    lambda x: lem(x))

Wall time: 3min 44s


Сгруппируем по имени и посчитаем сумму употреблений.

In [17]:
lem_names = lem_names.groupby(by='name').sum().reset_index()

In [18]:
lem_names.sort_values('count', ascending=False)

,name,count
119,Маугли,173
13,Багира,107
90,Котик,86
15,Бал,82
168,Рикки-Тикки,70
...,...,...
91,Котиковый Народ,1
93,Крик Пришелец,1
94,Куттар Гуджа,1
96,Кханхивар,1


### Вывод:
Мы получили 220 лемматизированных имен собственных и частоту их употреблений. Имена, состоящие из двух и более слов (Котиковый Народ), указаны корректно. 

# 2. Топ-100 самых частых слов

Необходимо нормализовать все слова в тексте. Лемматизация в данном случае займет слишком много времени. Решено использовать стемминг. 

Разделим текст на отдельные слова, оставим только буквы и дефисы. 

In [19]:
splited_text = re.sub(r'[^а-яА-я-]', ' ', text).split() 

Загрузим русские стоп-слова из библиотеки NLTK.

In [20]:
stop_words = set(nltk_stopwords.words('russian'))

Создадим таблицу для слов, стемматизированных слов и количества употреблений.

In [21]:
words = pd.DataFrame(columns=['word', 'stemmed', 'count'])

Напишем функцию для стемматизации слова.

In [22]:
stemmer = SnowballStemmer("russian")

def stemmed(word):
    stem = stemmer.stem(word)
    "".join(stem) 
    stemmed_word = "".join(stem)    
    return stemmed_word

Стемматизируем стоп-слова. 

In [23]:
stem_stop_words = list(stop_words)
for i in range(len(stem_stop_words)):
    stem_stop_words[i] = stemmed(stem_stop_words[i])

Напишем цикл, который будет проверять каждое слово на отсутствие в списке стоп-слов и создавать запись в таблице words, если условие выполнено. 

In [24]:
%%time
for i in range(len(splited_text)):
    w = stemmed(splited_text[i])
    if w not in stem_stop_words:
        words.loc[len(words)] = [splited_text[i].lower(), w, 1]

Wall time: 1min 45s


In [25]:
words

,word,stemmed,count
0,предисловие,предислов,1
1,автора,автор,1
2,сочинение,сочинен,1
3,рода,род,1
4,требует,треб,1
...,...,...,...
24701,нам,нам,1
24702,получать,получа,1
24703,приказания,приказан,1
24704,нашего,наш,1


Удалим лишние колонки и сгруппируем таблицу по стемматизированным словам. 

In [26]:
stemmed_words = words.drop(words.columns[0], axis=1).groupby(by='stemmed').sum().sort_values('count', ascending=False).reset_index()
stemmed_words.head(10)

,stemmed,count
0,сказа,292
1,маугл,228
2,котор,203
3,котик,164
4,наг,153
5,маленьк,137
6,слон,134
7,багир,124
8,одн,120
9,голов,118


Выберем топ-100 самых употрибляемых слов. 

In [27]:
top100_words = stemmed_words[:100]
top100_words.head(10)

,stemmed,count
0,сказа,292
1,маугл,228
2,котор,203
3,котик,164
4,наг,153
5,маленьк,137
6,слон,134
7,багир,124
8,одн,120
9,голов,118


### Вывод
Мы получили 100 самых употребляемых слова в стемматизированной форме и их частотность в тексте. В список слов не попали предлоги, частицы, местоимения и т.д..

# 3. Поиск всех форм самых часто используемых слов

Необходимо создать таблицу со словами и списками их форм.

Создадим таблицу и заполним колонку words пустыми списками. 

In [28]:
top10_words = stemmed_words[:10]
top10_words.columns = ['stemmed', 'words']
top10_words['words'] = ''
for i in range(10):
    top10_words.at[i, 'words'] = []
top10_words

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,stemmed,words
0,сказа,[]
1,маугл,[]
2,котор,[]
3,котик,[]
4,наг,[]
5,маленьк,[]
6,слон,[]
7,багир,[]
8,одн,[]
9,голов,[]


Напишем цикл, который соберет все формы стемматизированных слов в тексте. 

In [29]:
%%time
for i in range(10):
    word = top10_words['stemmed'][i]
    q = words[words['stemmed'] == word].reset_index(drop=True)
    for n in range(len(q)):
        if q['word'][n].lower() not in top10_words['words'][i]:
            top10_words['words'][i] = top10_words['words'][i] + list([q['word'][n].lower()])

Wall time: 95 ms


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\magics\execution.py:1315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code, glob, local_ns)


Выведем полученную таблицу в удобном формате. 

In [30]:
pd.set_option('max_colwidth', 130)
top10_words

,stemmed,words
0,сказа,"[сказал, сказанные, сказала, сказать, сказали, сказав, сказано, сказало]"
1,маугл,[маугли]
2,котор,"[который, которых, которая, которому, которое, которого, которые, которую, которой, которым, котором]"
3,котик,"[котик, котиков, котика, котики, котиком, котикам, котику]"
4,наг,"[наг, нага, нагены, нагом, нагена, наге, нагов, нагу]"
5,маленьк,"[маленького, маленькая, маленький, маленьких, маленькое, маленьким, маленьком, маленькой, маленькие, маленькому, маленькими]"
6,слон,"[слона, слонах, слон, слоны, слону, слоном, слонов, слонами, слоних]"
7,багир,"[багира, багиру, багирой, багиры, багире]"
8,одн,"[одному, одним, одну, одного, одно, одной, одни, одна, одном, одних, одними]"
9,голов,"[голова, головы, головой, голову, голове, головами]"


### Вывод:
Мы получили топ-10 самых часто используемых слов в тексте. Также для каждого слова собран список всех возможных форм.